# NarrativeKoGPT2 학습

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동.  
- 좌측상단 메뉴에서 런타임-> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장

### 1.1 GPU 연동 확인

In [53]:
!nvidia-smi

Wed Nov 11 17:31:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.09       Driver Version: 457.09       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2060   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   44C    P8     6W /  N/A |   5632MiB /  6144MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1.2 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

**Colab 디렉토리 아래 NarrativeKoGPT2 경로 확인**

**필요 패키지들 설치**

In [54]:
import os

import sys

print(os.getcwd())


C:\Users\multicampus\ssafy_pjt\TextGeneration\narrativeKoGPT2


## 2.KoGPT2 Transfer Learning

### 2.1.Import Package

In [55]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from kogpt2.utils import get_tokenizer
from kogpt2.utils import download, tokenizer
from model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from util.data import NovelDataset
import gluonnlp
from tqdm import tqdm

**torch GPU 확인**

In [56]:
print(torch.cuda.device_count())  # GPU deviec count check

1


In [57]:
import sentencepiece
import mxnet
import transformers
gluonnlp.__version__
mxnet.__version__
transformers.__version__

'2.4.1'

### 2.2. koGPT-2 Config

In [58]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch =100  # 학습 epoch
save_path = 'checkpoints/'
#use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000,
}

### 2.3 Model and Vocab Download

In [59]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

using cached model
using cached model


### 2.4.KoGPT-2 Model Vocab

In [60]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
# model_path로부터 다운로드 받은 내용을 load_state_dict으로 업로드
kogpt2model.load_state_dict(torch.load(model_path))

device = torch.device(ctx)
kogpt2model.to(device)

# kogpt2model.eval()
# 추가로 학습하기 위해 .train() 사용
kogpt2model.train()
vocab_b_obj = gluonnlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')


RuntimeError: Error(s) in loading state_dict for GPT2LMHeadModel:
	size mismatch for transformer.wpe.weight: copying a param with shape torch.Size([1024, 768]) from checkpoint, the shape in current model is torch.Size([512, 768]).
	size mismatch for transformer.h.0.attn.bias: copying a param with shape torch.Size([1, 1, 1024, 1024]) from checkpoint, the shape in current model is torch.Size([1, 1, 512, 512]).
	size mismatch for transformer.h.1.attn.bias: copying a param with shape torch.Size([1, 1, 1024, 1024]) from checkpoint, the shape in current model is torch.Size([1, 1, 512, 512]).
	size mismatch for transformer.h.2.attn.bias: copying a param with shape torch.Size([1, 1, 1024, 1024]) from checkpoint, the shape in current model is torch.Size([1, 1, 512, 512]).
	size mismatch for transformer.h.3.attn.bias: copying a param with shape torch.Size([1, 1, 1024, 1024]) from checkpoint, the shape in current model is torch.Size([1, 1, 512, 512]).
	size mismatch for transformer.h.4.attn.bias:

### 2.5. Get Batch Data using DataLoader

In [ ]:
tok_path = get_tokenizer()
model, vocab = kogpt2model, vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path)

#os.chdir("../")
data_file_path = 'Dataset/untokenized_Grimm.txt'
batch_size = 1
novel_dataset = NovelDataset(data_file_path, vocab, sentencepieceTokenizer)
novel_data_loader = DataLoader(novel_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

### 2.6. Learning rate, Loss function, Adam Optimizer

In [ ]:
learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.
    
    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map

### 2.7. KoGPT-2 Transfer Laerning

In [ ]:
print('KoGPT-2 Transfer Learning Start')
epoch = 100
for epoch in range(epoch):
  count = 0
  for data in novel_data_loader:
    optimizer.zero_grad()
    data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
    
    data= data.transpose(1,0)
    data= data.to(ctx)
    
    outputs = model(data, labels=data)
    loss, logits = outputs[:2]
    loss.backward()
    
    optimizer.step()
    if count %10 ==0:
      print('epoch no.{} train no.{}  loss = {}' . format(epoch, count+1, loss))
      # torch.save(model,save_path+'checkpoint_{}_{}.tar'.format(epoch,count))
      # 추론 및 학습 재개를 위한 일반 체크포인트 저장하기
    if (count >0 and count%100==0) or epoch % 5 == 0:
      torch.save({
        'epoch': epoch,
        'train_no': count,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':loss
      }, save_path+'narrativeKoGPT2_checkpoint.tar')
      print('saved')

    count += 1